In [0]:
%tensorflow_version 1.x

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import cv2
import imp
import numpy as np
import os         
from random import shuffle 
from tqdm import tqdm
import skimage as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [4]:
!unzip "drive/My Drive/TreeClassification/train.zip"

Archive:  drive/My Drive/TreeClassification/train.zip
   creating: train/
  inflating: train/Ashoka1(10).png   
  inflating: train/Ashoka1(11).png   
  inflating: train/Ashoka1(12).png   
  inflating: train/Ashoka1(13).png   
  inflating: train/Ashoka1(14).png   
  inflating: train/Ashoka1(15).png   
  inflating: train/Ashoka1(16).png   
  inflating: train/Ashoka1(17).png   
  inflating: train/Ashoka1(18).png   
  inflating: train/Ashoka1(19).png   
  inflating: train/Ashoka1(20).png   
  inflating: train/Ashoka1(21).png   
  inflating: train/Ashoka1(22).png   
  inflating: train/Ashoka1(23).png   
  inflating: train/Ashoka1(24).png   
  inflating: train/Ashoka1(25).png   
  inflating: train/Ashoka1(26).png   
  inflating: train/Ashoka1(27).png   
  inflating: train/Ashoka1(28).png   
  inflating: train/Ashoka1(37).png   
  inflating: train/Ashoka1(38).png   
  inflating: train/Ashoka1(39).png   
  inflating: train/Ashoka1(40).png   
  inflating: train/Ashoka1(41).png   
  inflating: t

In [5]:
!unzip "drive/My Drive/TreeClassification/test.zip"

Archive:  drive/My Drive/TreeClassification/test.zip
   creating: test/
  inflating: test/Ashoka1(1).png     
  inflating: test/Ashoka1(2).png     
  inflating: test/Ashoka1(3).png     
  inflating: test/Ashoka1(4).png     
  inflating: test/Ashoka1(5).png     
  inflating: test/Ashoka1(6).png     
  inflating: test/Ashoka1(7).png     
  inflating: test/Camellia(13).png   
  inflating: test/Camellia(2).png    
  inflating: test/Camellia(23).png   
  inflating: test/Camellia(30).png   
  inflating: test/Camellia(33).png   
  inflating: test/Camellia(43).png   
  inflating: test/Camellia(50).png   
  inflating: test/Chirstmas tree(12).png  
  inflating: test/Chirstmas tree(13).png  
  inflating: test/Chirstmas tree(14).png  
  inflating: test/Chirstmas tree(2).png  
  inflating: test/Chirstmas tree(3).png  
  inflating: test/Chirstmas tree(4).png  
  inflating: test/Cycas revoluta(12).png  
  inflating: test/Cycas revoluta(2).png  
  inflating: test/Cycas revoluta(22).png  
  inflating: 

In [0]:
# Path of plant images for training
TRAIN_DIR = "train"
TEST_DIR = "test"

In [0]:

# Input Image Size
IMG_SIZE = 28

In [8]:
plant_species = ['Ashoka1', 'Camellia', 'Chirstmas tree', 'Cycas revoluta', 'Guava', 'Jarul', 'Laurel Magnolia',
                 'Litchi', 'Madanmast', 'Mango', 'Mehndi', 'Rose', 'Rubbertree', 'Silver oak']

num_classes = len(plant_species)
print('No of species to be classified = ', num_classes)

species_list = sorted(plant_species)
print('List of Species = \n', species_list)

No of species to be classified =  14
List of Species = 
 ['Ashoka1', 'Camellia', 'Chirstmas tree', 'Cycas revoluta', 'Guava', 'Jarul', 'Laurel Magnolia', 'Litchi', 'Madanmast', 'Mango', 'Mehndi', 'Rose', 'Rubbertree', 'Silver oak']


In [0]:
# Horizontal Flip
def horizontal_flip(img_data):
	im_hFlip = np.fliplr(img_data)
	return im_hFlip

In [0]:
# Vertical Flip
def vertical_flip(img_data):
	im_vFlip = np.flipud(img_data)
	return im_vFlip

In [0]:
# Image Rotation (90 deg anti-clockwise)
def rotate_90(img_data):
	im_Rot90 = np.rot90(img_data, k=1, axes=(0,1))
	return im_Rot90

In [0]:
def create_label(img):
    print (img)
    word_label = img.split('(')[0]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'Ashoka1': return [1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    #                             [no cat, very doggo]
    elif word_label == 'Camellia': return [0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Chirstmas tree': return [0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Cycas revoluta': return [0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Guava': return [0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Jarul': return [0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif word_label == 'Laurel Magnolia': return [0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif word_label == 'Litchi': return [0,0,0,0,0,0,0,1,0,0,0,0,0,0]
    elif word_label == 'Madanmast': return [0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif word_label == 'Mango': return [0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif word_label == 'Mehndi': return [0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif word_label == 'Rose': return [0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif word_label == 'Rubbertree': return [0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif word_label == 'Silver oak': return [0,0,0,0,0,0,0,0,0,0,0,0,0,1]

In [0]:
# Image Rotation (90 def clockwise)
def rotate_270(img_data):
	im_Rot270 = np.rot90(img_data, k=3, axes=(0,1))
	return im_Rot270

In [0]:
# Random Noise
def rand_noise(img_data):
	im_randNoise = sk.util.random_noise(img_data)
	return im_randNoise

In [0]:
# Method to create training data
def create_train_data():
	training_data = []
	for img in tqdm(os.listdir(TRAIN_DIR)):
		path = os.path.join(TRAIN_DIR, img)
		img_data = cv2.imread(path)
		img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))

		# Image Augmentation
		img_data_hFlip = horizontal_flip(img_data)
		img_data_vFlip = vertical_flip(img_data)
		img_data_Rot90 = rotate_90(img_data)
		img_data_Rot270 = rotate_270(img_data)
		img_data_RandNoise = rand_noise(img_data)

		img_label = create_label(img)
		training_data.append([np.array(img_data), img_label])
		training_data.append([np.array(img_data_hFlip), img_label])
		training_data.append([np.array(img_data_vFlip), img_label])
		training_data.append([np.array(img_data_Rot90), img_label])
		training_data.append([np.array(img_data_Rot270), img_label])
		training_data.append([np.array(img_data_RandNoise), img_label])
    
	shuffle(training_data)
	return training_data

In [0]:
# Method to create testing data
def create_test_data():
	testing_data = []
	for img in tqdm(os.listdir(TEST_DIR)):
		path = os.path.join(TEST_DIR, img)
		img_data = cv2.imread(path)
		img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
		img_label = create_label(img)
		testing_data.append([np.array(img_data), img_label])
	shuffle(testing_data)
	return testing_data

In [17]:
# Create training data
train_data = create_train_data()

 13%|█▎        | 76/576 [00:00<00:01, 299.09it/s]

Litchi(45).png
Litchi(20).png
Ashoka1(18).png
Madanmast(13).png
Rubbertree(19).png
Jarul(29).png
Litchi(6).png
Chirstmas tree(50).png
Mango(3).png
Rose(15).png
Rose(23).png
Jarul(45).png
Silver oak(29).png
Ashoka1(53).png
Litchi(5).png
Laurel Magnolia(25).png
Rubbertree(29).png
Rose(19).png
Cycas revoluta(21).png
Silver oak(14).png
Chirstmas tree(7).png
Ashoka1(38).png
Jarul(7).png
Ashoka1(13).png
Silver oak(12).png
Guava(46).png
Camellia(31).png
Litchi(29).png
Rose(26).png
Laurel Magnolia(48).png
Rubbertree(5).png
Chirstmas tree(16).png
Mango(21).png
Silver oak(25).png
Rubbertree(9).png
Camellia(21).png
Guava(3).png
Guava(23).png
Jarul(15).png
Guava(8).png
Laurel Magnolia(50).png
Chirstmas tree(31).png
Cycas revoluta(31).png
Guava(34).png
Ashoka1(17).png
Jarul(36).png
Silver oak(38).png
Litchi(26).png
Madanmast(24).png
Camellia(34).png
Mehndi(19).png
Jarul(1).png
Cycas revoluta(6).png
Camellia(45).png
Laurel Magnolia(4).png
Chirstmas tree(45).png
Rubbertree(20).png
Mango(23).png
Rubbe

 30%|███       | 173/576 [00:00<00:01, 369.39it/s]

Chirstmas tree(6).png
Rose(37).png
Camellia(41).png
Rubbertree(32).png
Guava(37).png
Mehndi(39).png
Litchi(13).png
Chirstmas tree(26).png
Guava(5).png
Silver oak(40).png
Cycas revoluta(27).png
Guava(11).png
Madanmast(44).png
Silver oak(19).png
Jarul(14).png
Ashoka1(49).png
Rubbertree(45).png
Guava(20).png
Litchi(4).png
Jarul(21).png
Madanmast(47).png
Laurel Magnolia(20).png
Ashoka1(16).png
Camellia(32).png
Cycas revoluta(45).png
Mehndi(1).png
Madanmast(35).png
Laurel Magnolia(9).png
Camellia(42).png
Silver oak(31).png
Ashoka1(28).png
Mango(27).png
Laurel Magnolia(28).png
Laurel Magnolia(47).png
Rose(18).png
Madanmast(39).png
Rose(29).png
Silver oak(44).png
Mehndi(14).png
Guava(29).png
Ashoka1(39).png
Madanmast(37).png
Litchi(40).png
Laurel Magnolia(18).png
Mango(29).png
Silver oak(15).png
Jarul(26).png
Chirstmas tree(34).png
Rubbertree(25).png
Mehndi(10).png
Camellia(17).png
Ashoka1(19).png
Mehndi(29).png
Ashoka1(24).png
Jarul(43).png
Chirstmas tree(48).png
Mango(26).png
Camellia(40).p

 48%|████▊     | 274/576 [00:00<00:00, 425.26it/s]

...

Mehndi(20).png
Jarul(11).png
Cycas revoluta(8).png
Laurel Magnolia(5).png
Jarul(44).png
Mehndi(17).png
Mehndi(11).png
Mehndi(28).png
Silver oak(9).png
Cycas revoluta(19).png
Cycas revoluta(20).png
Litchi(7).png
Cycas revoluta(48).png
Laurel Magnolia(31).png
Madanmast(36).png
Camellia(19).png
Litchi(1).png
Guava(47).png
Litchi(50).png
Rubbertree(47).png
Chirstmas tree(23).png
Cycas revoluta(24).png
Chirstmas tree(11).png
Chirstmas tree(35).png
Litchi(11).png
Ashoka1(9).png
Cycas revoluta(36).png
Madanmast(50).png
Guava(10).png
Mango(4).png
Ashoka1(27).png
Litchi(10).png
Rubbertree(6).png
Mehndi(24).png
Laurel Magnolia(37).png
Chirstmas tree(39).png
Jarul(50).png
Mango(15).png
Rubbertree(42).png
Madanmast(26).png
Mehndi(38).png
Litchi(18).png
Madanmast(11).png
Chirstmas tree(46).png
Camellia(18).png
Guava(45).png
Madanmast(30).png
Madanmast(5).png
Rubbertree(49).png
Madanmast(6).png
Camellia(46).png
Mehndi(9).png
Chirstmas tree(36).png
Cycas revoluta(41).png
Guava(6).png
Ashoka1(54).png


 84%|████████▍ | 483/576 [00:01<00:00, 493.03it/s]

Cycas revoluta(11).png
Litchi(46).png
Madanmast(49).png
Madanmast(25).png
Mango(7).png
Rose(16).png
Guava(38).png
Madanmast(23).png
Camellia(6).png
Silver oak(18).png
Ashoka1(51).png
Rose(14).png
Rose(28).png
Litchi(43).png
Mehndi(36).png
Jarul(25).png
Guava(35).png
Cycas revoluta(42).png
Rubbertree(41).png
Jarul(24).png
Camellia(11).png
Jarul(35).png
Chirstmas tree(29).png
Madanmast(41).png
Chirstmas tree(28).png
Camellia(48).png
Guava(50).png
Guava(18).png
Chirstmas tree(9).png
Chirstmas tree(18).png
Cycas revoluta(26).png
Litchi(49).png
Rubbertree(22).png
Mehndi(21).png
Ashoka1(59).png
Mango(20).png
Cycas revoluta(1).png
Mango(9).png
Rubbertree(17).png
Cycas revoluta(35).png
Laurel Magnolia(43).png
Guava(30).png
Mehndi(16).png
Rose(9).png
Mango(19).png
Laurel Magnolia(36).png
Mango(34).png
Guava(36).png
Madanmast(46).png
Jarul(48).png
Litchi(24).png
Jarul(18).png
Chirstmas tree(37).png
Cycas revoluta(13).png
Mehndi(31).png
Madanmast(17).png
Madanmast(1).png
Silver oak(32).png
Rose(2

100%|██████████| 576/576 [00:01<00:00, 478.54it/s]

Jarul(13).png
Laurel Magnolia(45).png
Laurel Magnolia(46).png
Silver oak(17).png
Ashoka1(20).png
Litchi(31).png
Jarul(8).png
Mango(28).png
Chirstmas tree(21).png
Laurel Magnolia(15).png
Mango(5).png
Ashoka1(11).png
Camellia(22).png
Chirstmas tree(1).png
Mango(6).png
Guava(39).png
Camellia(3).png
Jarul(30).png
Silver oak(27).png
Laurel Magnolia(29).png
Camellia(39).png
Jarul(40).png
Guava(21).png
Mango(18).png
Litchi(39).png
Guava(44).png
Rose(20).png
Ashoka1(41).png
Rubbertree(37).png
Silver oak(7).png
Rose(34).png
Cycas revoluta(17).png
Camellia(27).png
Ashoka1(26).png
Guava(1).png
Litchi(19).png
Camellia(16).png
Guava(13).png
Chirstmas tree(33).png
Jarul(38).png
Mango(24).png
Litchi(15).png
Laurel Magnolia(41).png
Mango(35).png
Mango(25).png
Guava(42).png
Laurel Magnolia(21).png
Jarul(31).png
Silver oak(6).png
Guava(19).png
Cycas revoluta(5).png
Silver oak(33).png
Rose(36).png
Rose(10).png
Chirstmas tree(40).png
Silver oak(34).png
Ashoka1(25).png
Ashoka1(8).png
Litchi(36).png
Rubbert

In [18]:
# Unroll training set data in vector form
x_train = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
print('Train_X = ', type(x_train), x_train.shape)


Train_X =  <class 'numpy.ndarray'> (3456, 28, 28, 3)


In [19]:
# Training labels
y_train = np.array([i[1] for i in train_data])
print('Train_Y = ', type(y_train), y_train.shape)

Train_Y =  <class 'numpy.ndarray'> (3456, 14)


In [20]:
# Create testing data
test_data = create_test_data()

100%|██████████| 76/76 [00:00<00:00, 605.87it/s]

Mehndi(32).png
Ashoka1(4).png
Mango(2).png
Chirstmas tree(14).png
Camellia(43).png
Mehndi(12).png
Ashoka1(5).png
Camellia(50).png
Rubbertree(3).png
Litchi(32).png
Jarul(23).png
Ashoka1(6).png
Cycas revoluta(2).png
Chirstmas tree(13).png
Guava(2).png
Silver oak(3).png
Rose(12).png
Laurel Magnolia(22).png
Jarul(2).png
Mango(22).png
Litchi(2).png
Madanmast(42).png
Litchi(12).png
Mango(12).png
Laurel Magnolia(2).png
Rubbertree(4).png
Mango(32).png
Camellia(2).png
Cycas revoluta(12).png
Chirstmas tree(2).png
Camellia(30).png
Mehndi(22).png
Cycas revoluta(32).png
Chirstmas tree(3).png
Rose(2).png
Rubbertree(13).png
Camellia(23).png
Mehndi(2).png
Madanmast(2).png
Silver oak(22).png
Ashoka1(7).png
Cycas revoluta(22).png
Cycas revoluta(3).png
Rubbertree(15).png
Jarul(33).png
Guava(22).png
Jarul(3).png
Silver oak(45).png
Ashoka1(1).png
Rose(32).png
Laurel Magnolia(12).png
Mango(33).png
Rose(22).png
Madanmast(32).png
Litchi(22).png
Ashoka1(2).png
Litchi(42).png
Silver oak(11).png
Rubbertree(14).p

In [21]:
# Unroll testing set data in vector form
x_test = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
print('Test_X = ', type(x_test), x_test.shape)

Test_X =  <class 'numpy.ndarray'> (76, 28, 28, 3)


In [22]:
# Testing labels
y_test = np.array([i[1] for i in test_data])
print('Test_Y = ', type(y_test), y_test.shape)

Test_Y =  <class 'numpy.ndarray'> (76, 14)
